In [1]:
#!/usr/bin/env python
# coding: utf-8

from itertools import chain
import math
import argparse
import json
import logging
import os
import time
os.chdir('/data/hyeryung/mucoco')
import numpy as np
import pandas as pd
import torch
import transformers
from transformers import AutoConfig, AutoModelForMaskedLM, AutoTokenizer

import new_module.losses as lossbuilder
import wandb
# from new_module.decode_utils import (
#     beam_rerank_v0,
#     beam_rerank_v1,
#     beam_rerank_v2,
#     combi_rerank,
# )
from new_module.new_decode_utils import get_beam_hypotheses, get_combi_hypotheses, final_reranking
from new_module.evaluate_wandb import evaluate_main
from new_module.locate.new_locate_utils import LocateMachine
from new_module.utils.robertacustom import RobertaCustomForSequenceClassification

logging.basicConfig(level=logging.DEBUG, format="%(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(os.environ.get("LOGGING_LEVEL", logging.DEBUG))


In [45]:
import importlib
import new_module.new_decode_utils
importlib.reload(new_module.new_decode_utils)
from new_module.new_decode_utils import get_beam_hypotheses, get_combi_hypotheses, final_reranking

In [2]:
import joblib
config = joblib.load('config.pkl')

In [3]:
config

{'task': 'toxicity',
 'source_data': 'new_module/data/toxicity-avoidance/dev_set.jsonl',
 'source_style': 'toxic',
 'target_style': 'nontoxic',
 'target_label_ids': [0, 0],
 'model_paths': ['gpt2-large',
  '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint'],
 'tokenizer_paths': ['gpt2-large',
  '/data/hyeryung/loc_edit/models/roberta-base-jigsaw-toxicity-classifier-energy-training/step_1000_best_checkpoint'],
 'model_types': ['AutoModelForCausalLM',
  'RobertaCustomForSequenceClassification'],
 'output_dir_prefix': 'outputs/toxicity/devset',
 'early_stopping_patience': 0,
 'method': 'mlm-beamsearch-v0',
 'locate_unit': 'word',
 'min_epsilons': [0.9],
 'num_samples': 10,
 'device': 'cuda',
 'target_type': 'embeds',
 'cache_dir': '/data/hyeryung/hf_cache',
 'jsonl_primary_key': 'prompt',
 'jsonl_secondary_key': 'text',
 'losses': ['gpt2', 'classification_no_prefix_logprobloss'],
 'build_loss_dict': {'coeff_steps': 200,
  'co

In [4]:
main_start_time = time.time()

if not config.get("model_tag", None):
    if "energy-training" in config["model_paths"][1]:
        config["model_tag"] = "em"
    else:
        config["model_tag"] = "clsf"

    if (config["task"] == "formality") and ("gyafc" in config["model_paths"][1]):
        config["model_tag"] += "-gyafc"

if config["resume"]:
    logger.info("resuming from a previous run")
    run = wandb.init(
        project=config["wandb_project"],
        entity=config["wandb_entity"],
        id=config["wandb_run_id"],
        resume="must",
    )
else:
    run = wandb.init(
        project=config["wandb_project"],
        entity=config["wandb_entity"],
        config=config,
    )

run_id = run.path.split("/")[-1]
display_name = f"{run_id}"


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Starting new HTTPS connection (1): api.wandb.ai:443
https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 1812
https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 377
wandb: Currently logged in as: hayleyson. Use `wandb login --relogin` to force relogin
Popen(['git', 'cat-file', '--batch-check'], cwd=/data/hyeryung/mucoco, stdin=<valid stream>, shell=False, universal_newlines=False)


In [5]:


outdir = os.path.join(config["output_dir_prefix"], display_name)
os.makedirs(outdir, exist_ok=True)
outfile = f"{outdir}/outputs_epsilon{config['min_epsilons'][0]}.txt"
run.summary["outfile_path"] = outfile


In [6]:

class dummyArgs:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

build_loss_args = dummyArgs(**config["build_loss_dict"])

## load data
if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    source_dataset = [
        json.loads(l)[config["jsonl_primary_key"]][config["jsonl_secondary_key"]]
        for l in open(config["source_data"])
    ]
    generation_dataset = [
        json.loads(l)["generations"] for l in open(config["source_data"])
    ]
elif (config["task"] == "formality") or (config["task"] == "sentiment-lewis-compr"):
    with open(config["source_data"], "r") as f:
        generation_dataset = [line.rstrip('\n') for line in f.readlines()]
    source_dataset = ["" for l in generation_dataset]

# check if outfile exists
if (config["resume"]) and (os.path.exists(outfile)):

    with open(outfile, "r") as f:
        existing_gens = [x.rstrip("\n") for x in f.readlines()]
    resume_idx = len(existing_gens)
    if resume_idx == len(source_dataset):
        logger.debug("output file is already complete. skipping this run.")
        raise
    elif resume_idx < len(source_dataset):
        logger.info(
            f"output file already exists but is incomplete. resuming from index: {resume_idx}"
        )
        outf = open(outfile, "a")
        int_outf = open(outfile+".intermediate", "a")
    else:
        logger.critical(
            f"output file seems to be corrupted. The file length is {resume_idx}, where the size of source_dataset is {len(source_dataset)}"
        )
        raise
else:
    resume_idx = 0
    outf = open(outfile, "w")
    int_outf = open(outfile+".intermediate", "w")


In [7]:

## load tokenizer, models, define losses
name2tokenizer = {}
name2model = {}
name2config = {}
loss2tokenizer = {}
embed_luts = []

for i, model_path in enumerate(config["model_paths"]):
    if (
        model_path not in name2model
    ):  # making sure we are not loading the model twice in case some constraints use the same model.
        try:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=True,
            )
        except:
            name2tokenizer[config["tokenizer_paths"][i]] = AutoTokenizer.from_pretrained(
                config["tokenizer_paths"][i],
                cache_dir=config["cache_dir"],
                use_fast=False,
            )

        name2config[model_path] = AutoConfig.from_pretrained(
            model_path, cache_dir=config["cache_dir"]
        )

        if config["model_types"][i] == "RobertaCustomForSequenceClassification":
            name2model[model_path] = lossbuilder.ModelWrapper(
                RobertaCustomForSequenceClassification.from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        else:
            name2model[model_path] = lossbuilder.ModelWrapper(
                getattr(transformers, config["model_types"][i]).from_pretrained(
                    model_path,
                    config=name2config[model_path],
                    cache_dir=config["cache_dir"],
                )
            )
        name2model[model_path].eval()
        name2model[model_path].to(config['device'])

    input_embeds = name2model[model_path].get_input_embeddings()
    if isinstance(input_embeds, torch.nn.Sequential):
        input_embeds = input_embeds[0]
    embed_luts.append(input_embeds)

    if config["target_type"] == "embeds":
        embed_luts[-1].requires_grad = False

mlm_tokenizer = AutoTokenizer.from_pretrained("roberta-base")
mlm = None if config["method"] == "mlm-beamsearch-v2" else AutoModelForMaskedLM.from_pretrained("roberta-base").to(config['device'])

lossfns = []
for i, loss in enumerate(config["losses"]):
    lossfns.append(
        lossbuilder.build_loss(
            loss,
            name2model[config["model_paths"][i]],
            name2tokenizer[config["tokenizer_paths"][i]],
            build_loss_args,
        )
    )
    lossfns[i].tokenizer.add_special_tokens({"mask_token": mlm_tokenizer.mask_token})
    loss2tokenizer[loss] = lossfns[i].tokenizer


Starting new HTTPS connection (1): huggingface.co:443
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /gpt2-large/resolve/main/generation_config.json HTTP/1.1" 200 0


50265


https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0


In [8]:

# define an object to locate problematic phrases
locator = LocateMachine(lossfns[1].model, lossfns[1].tokenizer)

label_ids = config["target_label_ids"]  # target label's ids for each loss

run.summary["prep_time"] = time.time() - main_start_time
## beginning of main logic
decode_start_time = time.time()
# text_id = 0
if config["resume"]:
    num_skipped = run.summary.get("num_skipped", 0)
    num_edited = run.summary.get("num_edited", 0)
    num_decoded_tokens = run.summary.get("num_decoded_tokens", 0)
else:
    num_skipped = 0
    num_edited = 0
    num_decoded_tokens = 0


In [83]:

loss_weights = [1 - wandb.config.closs_weight, wandb.config.closs_weight]
interrupted = False
# for text_id in range(len(source_dataset))[resume_idx:]:
text_id = 0
source_text = source_dataset[text_id]
if source_text == "":
    source_text = lossfns[0].tokenizer.bos_token

if (config["task"] == "toxicity") or (config["task"] == "sentiment"):
    AR_prediction_all = [x["text"] for x in generation_dataset[text_id]]
    # predicted_batches = [x["tokens"] for x in generation_dataset[text_id]]
    # predicted_batches = [
    #     torch.tensor([x], dtype=torch.long, device=config["device"])
    #     for x in predicted_batches
    # ]
    
elif (config["task"] == "formality") or (
    config["task"] == "sentiment-lewis-compr"
):
    AR_prediction_all = [generation_dataset[text_id]]

curr_num_samples = len(AR_prediction_all)


In [84]:
AR_prediction_all

[' dirt. Unfortunately the majority of horses end up in shit that you had to drive yourself. My only recourse is to feed it myself. What would be the happy tale of my life then?',
 ' wearing games and holy ****ing shit do I hate horse wearing games.',
 " fetishes: it just makes me want to puke every time I see it on the internet, even though it's not worth a thing because I am a furry."]

In [148]:
# for sample_idx in range(config["num_samples"])[:]:

######### change here! instead of for loop, do a batched operation ########

# --------------------------------------------------------------------------------------------- #
## check whether initial text satisfies constraint

curr_loss = torch.zeros(len(AR_prediction_all)).to(config['device'])
logging_loss = torch.zeros((len(AR_prediction_all),2)).to(config['device'])
edit_yn = torch.ones(len(AR_prediction_all), dtype=torch.bool).to(config['device'])
        
for lossid, lossname in enumerate(config["losses"]):
    with torch.no_grad():
        lossvalue = lossfns[lossid].compute_gold_loss(
            source_text, AR_prediction_all,
            label_id=config['target_label_ids'][lossid],
        )
        torch.cuda.empty_cache()
    curr_loss += loss_weights[lossid] * lossvalue
    logging_loss[:, lossid] = lossvalue.clone()



In [149]:
allsat = logging_loss[:,1] < -math.log(config["min_epsilons"][0])
allsat_ix = allsat.nonzero().squeeze(0)
edit_yn[allsat_ix] = 0
edited_at_all_yn = edit_yn.detach().clone()


In [150]:
allsat, allsat_ix, edit_yn, edited_at_all_yn

(tensor([False, False, False], device='cuda:0'),
 tensor([], device='cuda:0', size=(0, 1), dtype=torch.int64),
 tensor([True, True, True], device='cuda:0'),
 tensor([True, True, True], device='cuda:0'))

In [88]:
if (edit_yn.sum().item() == 0) and (not config["dont_skip_allsat"]):
    ## save data
    num_edited += 0
    num_skipped += len(AR_prediction_all)
    num_decoded_tokens += 0
    print('continue')


In [89]:
    
num_edited += edit_yn.sum().item()
num_skipped += (len(AR_prediction_all) - edit_yn.sum().item())
num_decoded_tokens += sum([len(x) for x in name2tokenizer[config["tokenizer_paths"][0]]([x for i, x in enumerate(AR_prediction_all) if edit_yn[i] == 1], add_special_tokens=False).input_ids])


In [93]:
from copy import deepcopy

In [125]:
es_patience_count = torch.zeros(len(AR_prediction_all),dtype=torch.long).to(config['device'])
best_allsat = allsat.detach().clone()
best_losses = logging_loss.detach().clone()
best_weighted_loss = curr_loss.detach().clone()            
running_text = best_text = deepcopy(AR_prediction_all)
int_output = [{} for _ in range(len(AR_prediction_all))]


In [126]:
_iter = 0
# for _iter in range(wandb.config.n_iter):
masked_text = locator.locate_main(running_text, 
                                    method = config['locate_method'], 
                                    max_num_tokens = wandb.config.num_edit_token_per_step, 
                                    unit = config['locate_unit'], 
                                    num_layer = -2, #penultimate
                                    label_id = config['target_label_ids'][0])


In [127]:
AR_prediction_all

[' dirt. Unfortunately the majority of horses end up in shit that you had to drive yourself. My only recourse is to feed it myself. What would be the happy tale of my life then?',
 ' wearing games and holy ****ing shit do I hate horse wearing games.',
 " fetishes: it just makes me want to puke every time I see it on the internet, even though it's not worth a thing because I am a furry."]

In [128]:

inputs = mlm_tokenizer(
    masked_text, return_tensors="pt", padding=True, truncation=True
)
inputs = inputs.to(config['device']) 
masked_sequence=inputs['input_ids']


In [129]:

## make predictions for the masked indices
with torch.no_grad():
    logits = mlm(**inputs).logits


In [130]:

special_token_ids = mlm_tokenizer.convert_tokens_to_ids(mlm_tokenizer.all_special_tokens)
logits[:, :, special_token_ids] = -float("inf")


## 여기에서 repeat이 안되도록 처리할 수 있나? # 아직은 잘 모르겠음



In [131]:

indices_in_mlm_tokens = (
    inputs.input_ids == mlm_tokenizer.mask_token_id
).nonzero(as_tuple=True)

## get top k tokens for each index
predicted_token_ids = torch.topk(
    logits[indices_in_mlm_tokens[0], indices_in_mlm_tokens[1], :],
    k=config['k_per_location'],
    dim=-1,
)


In [132]:

if config["method"] in ["mlm-beamsearch-v0","mlm-beamsearch-v1"] :
    hypotheses = get_beam_hypotheses(source_text, 
            masked_sequence, 
            indices_in_mlm_tokens,
            predicted_token_ids.indices,
            mlm_tokenizer, 
            lossfns,
            config)
elif config["method"] == "mlm-reranking":
    hypotheses = get_combi_hypotheses(masked_sequence, 
                indices_in_mlm_tokens,
                predicted_token_ids.indices,
                mlm_tokenizer,
                config)


In [133]:
hypotheses

[[' dirt. Unfortunately the majority of you end up in something that you had to get to. My only recourse is to do it myself. What would be the happy tale of my life then?',
  ' dirt. Unfortunately the majority of you end up in something that you had to work in, My only recourse is to make it myself. What would be the happy tale of my life then?',
  ' dirt. Unfortunately the majority of you end up in something that you had to go for." My only recourse is to face it myself. What would be the happy tale of my life then?',
  ' dirt. Unfortunately the majority of you end up in something that you had to help with? My only recourse is to live it myself. What would be the happy tale of my life then?',
  ' dirt. Unfortunately the majority of you end up in something that you had to deal from! My only recourse is to buy it myself. What would be the happy tale of my life then?'],
 [' wearing games and by ****ing god do I hate horse wearing games.',
  ' wearing games and no ****ing why do I hate ho

In [134]:
    
final_hypotheses, new_best_weighted_loss, new_best_allsat, new_best_logging_loss = final_reranking(source_text,
                                                                                                   hypotheses,
                                                                                                    lossfns,
                                                                                                    config,
                                                                                                     batch_size=64)


In [135]:
final_hypotheses

[' dirt. Unfortunately the majority of you end up in something that you had to get to. My only recourse is to do it myself. What would be the happy tale of my life then?',
 ' wearing games and no ****ing why do I hate horse wearing games.',
 " fetishes: it just makes me want to be better every time I see it on the news, even though it's not worth a thing because I am the.."]

In [136]:
new_best_weighted_loss

tensor([13.857,  5.988, 11.103], device='cuda:0')

In [137]:
new_best_allsat

tensor([ True, False,  True], device='cuda:0')

In [138]:
new_best_logging_loss

tensor([[  138.467,     0.011],
        [   54.674,     0.579],
        [  110.782,     0.027]], device='cuda:0')

In [139]:
new_best_allsat

tensor([ True, False,  True], device='cuda:0')

In [140]:

update = torch.Tensor([]).long().to(config['device'])
if wandb.config.selection_criteria == "weighted_sum":
    update = best_weighted_loss > new_best_weighted_loss
elif wandb.config.selection_criteria == "allsat_primary":
    update = (~best_allsat & new_best_allsat) | \
            (~best_allsat & ~new_best_allsat & (best_weighted_loss > new_best_weighted_loss)) | \
            (best_allsat & new_best_allsat & (best_losses[:, 0] > new_best_logging_loss[:, 0])) 


In [167]:

## intermediate output for debugging
for sample_ix in range(len(AR_prediction_all)):
    
    int_output[sample_ix].update({f"iter{_iter}_original_sentence": running_text[sample_ix],
                                f"iter{_iter}_masked_sentence": masked_text[sample_ix],
                                f"iter{_iter}_best_text": final_hypotheses[sample_ix],
                                f"iter{_iter}_update": update[sample_ix].item()})


In [168]:
int_output

[{'iter0_original_sentence': ' dirt. Unfortunately the majority of you end up in something that you had to get to. My only recourse is to do it myself. What would be the happy tale of my life then?',
  'iter0_masked_sentence': ' dirt. Unfortunately the majority of<mask> end up in<mask> that you had to<mask><mask><mask> My only recourse is to<mask> it myself. What would be the happy tale of my life then?',
  'iter0_best_text': ' dirt. Unfortunately the majority of you end up in something that you had to get to. My only recourse is to do it myself. What would be the happy tale of my life then?',
  'iter0_update': True},
 {'iter0_original_sentence': ' wearing games and no ****ing why do I hate horse wearing games.',
  'iter0_masked_sentence': ' wearing games and<mask> ****ing<mask> do I hate horse wearing games.',
  'iter0_best_text': ' wearing games and no ****ing why do I hate horse wearing games.',
  'iter0_update': True},
 {'iter0_original_sentence': " fetishes: it just makes me want 

In [143]:

running_text = deepcopy(final_hypotheses)
for update_index in update.nonzero().squeeze().tolist():
    best_text[update_index] = final_hypotheses[update_index]
best_allsat[update] = new_best_allsat[update]
best_losses[update] = new_best_logging_loss[update]
best_weighted_loss[update] = new_best_weighted_loss[update]


In [144]:
es_patience_count

tensor([0, 0, 0], device='cuda:0')

In [151]:
(best_allsat & edit_yn).nonzero().squeeze()

tensor([0, 2], device='cuda:0')

In [152]:
es_patience_count[(best_allsat & edit_yn).nonzero().squeeze()]

tensor([0, 0], device='cuda:0')

In [153]:

es_patience_count[(best_allsat & edit_yn).nonzero().squeeze()] += 1



In [154]:
es_patience_count

tensor([1, 0, 1], device='cuda:0')

In [155]:
config["early_stopping_patience"]

0

In [156]:
if (config["early_stopping_patience"] != -1):
    edit_yn[es_patience_count > config['early_stopping_patience']] = 0

In [157]:
edit_yn

tensor([False,  True, False], device='cuda:0')

In [158]:
edit_yn.sum()

tensor(1, device='cuda:0')

In [160]:
if (config["early_stopping_patience"] != -1):
    edit_yn[es_patience_count > config['early_stopping_patience']] = 0
if edit_yn.sum() == 0:
    print('break')

In [169]:

output = {
        "prompt": {
            "text": source_text,
        },
        "generations": [
            {
                "text": best_text[i],
                "original_text": AR_prediction_all[i],
                "allsat": best_allsat[i].item(),
                "losses": best_losses[i,:].tolist(),
                "weighted_loss": best_weighted_loss[i].item(),
                "edited": edited_at_all_yn[i].tolist(),
            } for i in range(len(AR_prediction_all))
        ],
    }
    
intermediate_output = {
        "prompt": {
            "text": source_text,
        },
        "generations": 
            int_output
        ,
    }


In [171]:
output

{'prompt': {'text': 'Let me just start by saying I hate horse'},
 'generations': [{'text': ' dirt. Unfortunately the majority of you end up in something that you had to get to. My only recourse is to do it myself. What would be the happy tale of my life then?',
   'original_text': ' dirt. Unfortunately the majority of horses end up in shit that you had to drive yourself. My only recourse is to feed it myself. What would be the happy tale of my life then?',
   'allsat': True,
   'losses': [138.46743774414062, 0.011374153196811676],
   'weighted_loss': 13.856980323791504,
   'edited': True},
  {'text': ' wearing games and no ****ing why do I hate horse wearing games.',
   'original_text': ' wearing games and holy ****ing shit do I hate horse wearing games.',
   'allsat': False,
   'losses': [54.674339294433594, 0.5787469744682312],
   'weighted_loss': 5.988306045532227,
   'edited': True},
  {'text': " fetishes: it just makes me want to be better every time I see it on the news, even tho

In [173]:

json.dump(output, outf)
outf.write("\n")
outf.flush()

json.dump(intermediate_output, int_outf)
int_outf.write("\n")
int_outf.flush()
        


In [174]:
if (time.time() - main_start_time) > config['server_time_limit'] * 60 * 60 * 0.9:
    interrupted = True
    print('break')

outf.close()
int_outf.close()


In [175]:

if config["resume"]:
    run.summary["decode_time"] += time.time() - decode_start_time
else:
    run.summary["decode_time"] = time.time() - decode_start_time
run.summary['num_decoded_tokens'] = num_decoded_tokens
run.summary['toks_p_sec'] = (num_decoded_tokens/run.summary['decode_time'])
run.summary["num_skipped"] = num_skipped
run.summary["num_edited"] = num_edited

run.finish()


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


decode_time,1319.89143
num_decoded_tokens,172
num_edited,6
num_skipped,0
outfile_path,outputs/toxicity/dev...
prep_time,17.00515
toks_p_sec,0.13031


Starting new HTTPS connection (1): o151352.ingest.sentry.io:443
https://o151352.ingest.sentry.io:443 "POST /api/4504800232407040/envelope/ HTTP/1.1" 200 0


In [ ]:



if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Locally Editing Text Generation")
    parser.add_argument(
        "--task",
        type=str,
        help="task name",
        choices=["toxicity", "formality", "sentiment", "sentiment-lewis-compr"],
    )
    parser.add_argument(
        "--source_data",
        type=str,
        default="data/formality/GYAFC_Corpus/Entertainment_Music/test/informal",
        help="source data path",
    )
    parser.add_argument(
        "--source_style", type=str, default="informal", help="source style"
    )
    parser.add_argument(
        "--target_style", type=str, default="formal", help="target style"
    )
    parser.add_argument(
        "--target_label_ids",
        nargs="+",
        type=int,
        default=[1, 1],
        help="a list of indices of target label used in each of models. e.g. [1,1]",
    )
    parser.add_argument(
        "--model_paths",
        nargs="+",
        type=str,
        default=[
            "gpt2-large",
            "/home/s3/hyeryung/data/loc_edit/roberta-base-pt16-formality-regressor-with-gpt2-large-embeds-rescale/epoch_17",
        ],
        help="model paths",
    )
    parser.add_argument(
        "--tokenizer_paths",
        nargs="+",
        type=str,
        default=[
            "gpt2-large",
            "/home/s3/hyeryung/data/loc_edit/roberta-base-pt16-formality-regressor-with-gpt2-large-embeds-rescale/epoch_17",
        ],
        help="tokenizer paths",
    )
    parser.add_argument(
        "--model_types",
        nargs="+",
        type=str,
        default=["AutoModelForCausalLM", "RobertaCustomForSequenceClassification"],
        help="model types",
    )
    parser.add_argument(
        "--output_dir_prefix",
        type=str,
        help="output directory prefix. e.g. outputs/formality/mlm-reranking",
    )
    parser.add_argument(
        "--early_stopping_patience",
        type=int,
        default=-1,
        help="early stopping patience",
    )
    parser.add_argument(
        "--method",
        type=str,
        default="mlm-beamsearch-v0",
        help="method name",
        choices=[
            "mlm-beamsearch-v0",
            "mlm-beamsearch-v1",
            "mlm-beamsearch-v2",
            "mlm-reranking",
        ],
    )
    parser.add_argument(
        "--locate_unit", type=str, default="token", help="unit to locate"
    )
    parser.add_argument(
        "--min_epsilons", nargs="+", type=float, default=[0.75], help="min epsilons"
    )
    parser.add_argument(
        "--num_samples",
        type=int,
        default=1,
        help="number of samples to edit per prompt",
    )
    parser.add_argument("--device", type=str, default="cuda", help="device")
    parser.add_argument(
        "--target_type",
        type=str,
        default="embeds",
        help="target type (embeds, simplex, probability) from prior work's code",
    )
    parser.add_argument(
        "--cache_dir", type=str, default="hf_cache", help="cache directory"
    )
    parser.add_argument(
        "--jsonl_primary_key", type=str, default="prompt", help="jsonl primary key"
    )
    parser.add_argument(
        "--jsonl_secondary_key", type=str, default="text", help="jsonl secondary key"
    )
    parser.add_argument(
        "--losses",
        nargs="+",
        type=str,
        default=["gpt2", "classification_no_prefix_logprobloss"],
        help="losses",
    )
    parser.add_argument(
        "--build_loss_dict",
        type=json.loads,
        default='{"coeff_steps": 200, "coeff_pattern": "constant", "loss_type": "xentropy", "length_normalize": false, "AR_temperature": 1.0, "AR_top_k": 0, "AR_top_p": 0.96, "max_output_length": 20}',
        help="build loss dict",
    )
    parser.add_argument(
        "--num_edit_token_per_step",
        type=int,
        default=5,
        help="number of edit tokens per step",
    )
    parser.add_argument("--k_per_location", type=int, default=15, help="k per location")
    parser.add_argument("--n_iter", type=int, default=3, help="number of iterations")
    parser.add_argument(
        "--selection_criteria",
        type=str,
        default="weighted_sum",
        help="selection criteria",
    )
    parser.add_argument("--closs_weight", type=float, default=0.32, help="closs weight")
    parser.add_argument("--beam_size", type=int, default=5, help="beam size")
    parser.add_argument(
        "--wandb_project", type=str, default="mlm_reranking", help="wandb project name"
    )
    parser.add_argument(
        "--wandb_entity", type=str, default="hayleyson", help="wandb entity name"
    )
    parser.add_argument("--wandb_run_id", type=str, help="wandb run name")
    parser.add_argument(
        "--resume", action="store_true", help="whether to resume from a previous run"
    )
    parser.add_argument("--slurm_job_id", type=str, help="slurm job id (for debugging)")
    parser.add_argument(
        "--dont_skip_allsat",
        action="store_true",
        help="if this argument is passed, the module will conduct decoding on all samples even if they already satisfy constraints",
    )
    parser.add_argument(
        "--locate_method",
        type=str,
        help="method to use for locating tokens",
        choices=["attention", "grad_norm"],
        default="attention",
    )
    parser.add_argument(
        "--server_time_limit",
        type=float,
        help="Number of maximum hours to run the script for. Can be fractions e.g. 7.5.",
        default=10000
    )

    args = parser.parse_args()
    config = vars(args)

    main(config)
